In [1]:
from dlhub_sdk.models.servables.python import PythonStaticMethodModel, PythonClassMethodModel
from dlhub_sdk.utils.schemas import validate_against_dlhub_schema
from dlhub_sdk.utils.types import compose_argument_block
from fair_research_login import NativeClient
from home_run.version import __version__
from unittest import TestCase
from tempfile import mkstemp
from platform import system
import pickle as pkl
import numpy as np
import requests
import json
import sys
import os

from home_run.python import PythonStaticMethodServable, PythonClassMethodServable

## Define a class to publish to DLHub

In [2]:
!cat file_length.py

def get_file_length(input_data):
    import os
    len_val = None
    print(input_data)

    # Check if it is a file
    try:
        if os.path.isfile(str(input_data)):
            len_val = os.path.getsize(str(input_data))
    except:
        pass

    # Otherwise, get the length of it
    if not len_val:
        len_val = len(input_data)

    return len_val


In [3]:
from file_length import get_file_length

In [4]:
print(get_file_length('Using-files.ipynb'))
print(get_file_length('aaa'))

Using-files.ipynb
10403
aaa
3


## Save the class and create a servable from it

Set the input type to 'file'

In [5]:
# Make the metadata file
model = PythonStaticMethodModel.create_model('file_length', 'get_file_length')
model.set_title('File Length Function')
model.set_name('FileLength')
model.set_domains(["general","test"])

model.add_files('file_length.py')

# Add provenance information
model.set_authors(["Chard, Ryan"], ["Argonne National Laboratory"])
model.set_abstract("A function to get the length of a file.")

# Set the input type to file
model.set_inputs('file', 'A file')
model.set_outputs('float', 'Output')

# Make the servable
servable = PythonStaticMethodServable(**model.to_dict())

## Get an auth token to test it

In [6]:
# Login and get a token for Petrel HTTPS
client = NativeClient(client_id='7414f0b4-7d05-4bb6-bb00-076fa3f17cf5')
tokens = client.login(requested_scopes=['https://auth.globus.org/scopes/56ceac29-e98a-440a-a594-b41e7a084b62/all'])
auth_token = tokens["petrel_https_server"]['access_token']
headers = {'Authorization': f'Bearer {auth_token}'}

Test that it works locally

In [7]:
servable.run({'url': 'https://45a53408-c797-11e6-9c33-22000a1e3b52.e.globus.org/ryan/README.txt',
              'headers': headers})

/var/folders/sr/vtk5zxk17t5bxzf2hkdkp1k00000gn/T/tmp457p1lc8/45a53408-c797-11e6-9c33-22000a1e3b52.e.globus.org/ryan/README.txt


381

## Publish the servable to DLHub

In [8]:
import dlhub_sdk
dl = dlhub_sdk.DLHubClient()

In [9]:
model.to_dict()

{'datacite': {'creators': [{'givenName': 'Ryan',
    'familyName': 'Chard',
    'affiliations': 'Argonne National Laboratory'}],
  'titles': [{'title': 'File Length Function'}],
  'publisher': 'DLHub',
  'publicationYear': '2019',
  'identifier': {'identifier': '10.YET/UNASSIGNED', 'identifierType': 'DOI'},
  'descriptions': [{'description': 'A function to get the length of a file.',
    'descriptionType': 'Abstract'}],
  'fundingReferences': [],
  'relatedIdentifiers': [],
  'alternateIdentifiers': [],
  'rightsList': [],
  'resourceType': {'resourceTypeGeneral': 'InteractiveResource'}},
 'dlhub': {'version': '0.8.0',
  'domains': ['general', 'test'],
  'visible_to': ['public'],
  'name': 'FileLength',
  'files': {'other': ['file_length.py']},
  'type': 'servable'},
 'servable': {'methods': {'run': {'input': {'type': 'file',
     'description': 'A file'},
    'output': {'type': 'float', 'description': 'Output'},
    'parameters': {},
    'method_details': {'method_name': 'get_file_len

In [10]:
task_id = dl.publish_servable(model)

In [11]:
task_id

'bc0b0c57-e877-49ad-a697-6bf1d723aa9b'

## Use the DLHub servable

In [12]:
df_serv = dl.search_by_servable(servable_name="filelength")
print(df_serv)
servable_name = df_serv[0]['dlhub']['shorthand_name']
print(servable_name)

[{'datacite': {'creators': [{'affiliations': 'Argonne National Laboratory', 'familyName': 'Chard', 'givenName': 'Ryan'}], 'descriptions': [{'description': 'A function to get the length of a file.', 'descriptionType': 'Abstract'}], 'identifier': {'identifier': '10.YET/UNASSIGNED', 'identifierType': 'DOI'}, 'publicationYear': '2019', 'publisher': 'DLHub', 'resourceType': {'resourceTypeGeneral': 'InteractiveResource'}, 'titles': [{'title': 'File Length Function'}]}, 'dlhub': {'build_location': '/mnt/dlhub_ingest/965df8fe-8ea4-4090-9542-ffa289d9ac2e-1560283549', 'domains': ['general', 'test'], 'ecr_arn': 'arn:aws:ecr:us-east-1:039706667969:repository/965df8fe-8ea4-4090-9542-ffa289d9ac2e', 'ecr_uri': '039706667969.dkr.ecr.us-east-1.amazonaws.com/965df8fe-8ea4-4090-9542-ffa289d9ac2e', 'files': {'other': ['file_length.py']}, 'id': '965df8fe-8ea4-4090-9542-ffa289d9ac2e', 'name': 'FileLength', 'owner': 'ryan_globusid', 'publication_date': '1560283549399', 'shorthand_name': 'ryan_globusid/FileLe

In [13]:
res = dl.run(servable_name, {'url': 'https://45a53408-c797-11e6-9c33-22000a1e3b52.e.globus.org/ryan/test-123.txt',
              'headers': headers}, input_type='json')

In [14]:
res

[381]